# DDoS Attacks Detection and Characterization


Import pyton libraries

In [3]:
import pandas as pd
import math
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('ddos_dataset.csv')


C:\Users\alber\AppData\Local\Temp\ipykernel_23344\2211235812.py:1: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ddos_dataset.csv')


## Section 1 – Data exploration and pre-processing

In [15]:
#Fix column names
df.columns = [col.strip() for col in df.columns]


27                    detectportal.firefox.com/success.txt
106                                     ocsp.digicert.com/
431                                   205.174.165.72/c.php
463                                   205.174.165.72/c.php
479                                   205.174.165.72/c.php
                               ...                        
29627                          talebi.ca/category/network/
29631       talebi.ca/wp-content/uploads/2017/09/logo2.png
36155    en.drshno.com/wp-content/uploads/2015/12/sa2-1...
36546    en.drshno.com/wp-content/uploads/2015/12/sa1-1...
37821    en.drshno.com/wp-content/themes/enfold/config-...
Name: SimillarHTTP, Length: 478, dtype: object


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64239 entries, 0 to 64238
Data columns (total 88 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               64239 non-null  int64  
 1   FlowID                   64239 non-null  object 
 2   SourceIP                 64239 non-null  object 
 3   SourcePort               64239 non-null  int64  
 4   DestinationIP            64239 non-null  object 
 5   DestinationPort          64239 non-null  int64  
 6   Protocol                 64239 non-null  int64  
 7   Timestamp                64239 non-null  object 
 8   FlowDuration             64239 non-null  int64  
 9   TotalFwdPackets          64239 non-null  int64  
 10  TotalBackwardPackets     64239 non-null  int64  
 11  TotalLengthofFwdPackets  64239 non-null  float64
 12  TotalLengthofBwdPackets  64239 non-null  float64
 13  FwdPacketLengthMax       64239 non-null  float64
 14  FwdPacketLengthMin    

In [17]:
df.describe()

,Unnamed: 0,SourcePort,DestinationPort,Protocol,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,...,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Inbound
count,64239.000000,64239.000000,64239.000000,64239.000000,6.423900e+04,64239.000000,64239.000000,64239.000000,6.423900e+04,64239.000000,...,6.423900e+04,6.423900e+04,6.423900e+04,6.423900e+04,6.423900e+04,6.423900e+04,6.423900e+04,6.423900e+04,6.423900e+04,64239.000000
mean,32637.819876,25267.368608,30715.252884,14.362864,5.150614e+06,18.814848,1.304036,6699.521630,2.153348e+03,613.992948,...,-4.167229e+07,9.026629e+03,4.878774e+03,1.687791e+04,6.204895e+03,1.155553e+06,2.613584e+05,1.555321e+06,8.908522e+05,0.919566
std,19008.473731,24710.413442,20347.990914,4.706001,2.249713e+07,462.305336,25.199720,20510.320389,9.087201e+04,585.691655,...,2.062767e+08,2.045563e+05,1.424082e+05,3.828583e+05,1.667756e+05,5.341380e+06,1.574889e+06,6.993415e+06,4.571276e+06,0.271966
min,0.000000,0.000000,0.000000,0.000000,1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,...,-1.062719e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,16209.500000,779.000000,12435.500000,17.000000,1.000000e+00,2.000000,0.000000,123.000000,0.000000e+00,53.000000,...,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
50%,32477.000000,29661.000000,30797.000000,17.000000,2.000000e+00,2.000000,0.000000,1050.000000,0.000000e+00,393.000000,...,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
75%,48726.500000,48510.500000,49041.500000,17.000000,2.087700e+04,2.000000,0.000000,2944.000000,0.000000e+00,1464.000000,...,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
max,65999.000000,65518.000000,65535.000000,17.000000,1.200000e+08,85894.000000,1862.000000,176000.000000,8.829488e+06,3564.000000,...,1.480000e+03,1.678654e+07,2.853681e+07,6.783473e+07,8.349846e+06,1.147455e+08,5.275036e+07,1.147455e+08,1.147455e+08,1.000000
